<a href="https://colab.research.google.com/github/deRockerTom/dataFrenchWords/blob/main/compressFrenchFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/deRockerTom/dataFrenchWords

Cloning into 'dataFrenchWords'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [51]:
import csv

# open csv
dic = {}
with open('dataFrenchWords/french-word-list-verbs.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

with open('dataFrenchWords/french-word-list-adjectives.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

with open('dataFrenchWords/french-word-list-nouns.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

dic = dict(sorted(dic.items(), key=lambda item: item[1], reverse=True))
dic = {k: dic[k] for k in list(dic)[:768]} # In order to code the dictionnary on only 2 other bytes with most frequent words
print(dic)

{'est': 1495122, 'ai': 1039767, 'autre': 311498, 'fai': 309755, 'peu': 258439, 'nouveau': 192744, 'grand': 191212, 'bon': 188301, 'petit': 161683, 'm?me': 133491, 'va': 123704, 'doi': 112301, 'di': 109837, 'dernier': 109357, 'voi': 96019, 'seul': 86629, 'pren': 80417, 'beau': 79318, 'sai': 74068, 'met': 73694, 'veu': 69787, 'jeune': 68814, 'meilleur': 65608, 'français': 64477, 'différent': 64457, 'trouv': 64184, 'nombreux': 63198, 'important': 61873, 'juste': 59079, 'présent': 58258, 'donn': 56499, 'ven': 56214, 'pass': 56037, 'fall': 54016, 'permett': 53302, 'vrai': 52402, 'part': 52271, 'politique': 50615, 'propre': 49986, 'social': 49961, 'possible': 49132, 'général': 48744, 'national': 47260, 'utilis': 46318, 'simple': 45294, 'plein': 44548, 'gratuit': 43640, 'gros': 43261, 'fort': 40137, 'pens': 39658, 'temps': 38463, 'haut': 38417, 'ancien': 38231, 'compren': 38010, 'rest': 37791, 'connai': 37008, 'international': 36420, 'jour': 36202, 'jou': 36179, 'principal': 36063, 'an': 3518

In [52]:
# class to build a tree defining all the possible  string in dic
class SuffixTree():
  class Node():
    def __init__(self,char):
      self.char = char
      self.out = {}
      self.isFinal = False
    def __str__(self, level=0):
      ret = "  " * level + str(self.char) + "\n"
      for c in self.out:
        ret += self.out[c].__str__(level + 1)
      return ret

  def __init__(self, dic):
    self.root = self.Node(None)
    self.root.isFinal = True
    for key in dic:
      cursor = self.root
      for c in key:
        if c in cursor.out:
          cursor = cursor.out[c]
        else:
          node = self.Node(c)
          cursor.out[c] = node
          cursor = node
      cursor.isFinal = True
  def longestPrefix(self, s):
    cursor = self.root
    word = ''
    temp = ''
    end = False
    i = 0
    length = len(s)
    while not(end) and i < length:
      if s[i] in cursor.out:
        temp = temp + s[i]
        cursor = cursor.out[s[i]]
        if cursor.isFinal:
          word = temp
        i += 1
      else:
        end = True
    return word
  def __str__(self):
    return f'{self.root}'

s = SuffixTree({'ab' : 1, 'abc': 2, 'coucou' : 5, 'aba' : 4, 'aaa' : 8})
print(s)
s.longestPrefix('abdc')
    


None
  a
    b
      c
      a
    a
      a
  c
    o
      u
        c
          o
            u



'ab'

In [ ]:
# First implementation

